In [2]:
import org.apache.spark.sql.{SparkSession, functions}
import org.apache.spark.sql.functions.{col, lit, sum}
import org.apache.spark

import org.apache.spark.sql.{SparkSession, functions}
import org.apache.spark.sql.functions.{col, lit, sum}
import org.apache.spark


In [3]:
val sparkSession = SparkSession
    .builder()
    .master("local[*]")
    .appName("curso")
    .config("spark.some.config.option", "some-value")
    .getOrCreate()

sparkSession: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@c15cb5a


In [4]:
var leer = sparkSession.read
    .option("header", "true")
    .option("sep", ";")
    .option("inferSchema", "true")
    .csv("Rango_Edades_Seccion_202203.csv")

leer: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


In [5]:
leer.show(false)

+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|DESC_DISTRITO       |COD_DIST_BARRIO|DESC_BARRIO         |COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|1           |CENTRO              |101            |PALACIO             |1         |1001            |1          |0           |3               |1               |1                 |null              |
|1           |CENTRO              |101            |PALACIO             |1         |1001            |1          |1           |5               |2               |1                 |null              |
|1        

In [6]:
val bar = leer.select(col("DESC_BARRIO")).distinct().show()

+--------------------+
|         DESC_BARRIO|
+--------------------+
|EMBAJADORES         |
|LEGAZPI             |
|DELICIAS            |
|CONCEPCION          |
|VENTAS              |
|IMPERIAL            |
|TRAFALGAR           |
|ESTRELLA            |
|ADELFAS             |
|PUEBLO NUEVO        |
|LOS CARMENES        |
|OPA�EL              |
|LOS JERONIMOS       |
|QUINTANA            |
|VALLEHERMOSO        |
|CANILLAS            |
|PALACIO             |
|ALMENDRALES         |
|CASTELLANA          |
|CORTES              |
+--------------------+
only showing top 20 rows



bar: Unit = ()


In [10]:
leer.createTempView("padron")

In [11]:
sparkSession.sql("SELECT COUNT(DISTINCT DESC_BARRIO) FROM padron").show()

+---------------------------+
|count(DISTINCT DESC_BARRIO)|
+---------------------------+
|                        132|
+---------------------------+



In [12]:
val lon = leer.withColumn("Longitud",functions.length(col("DESC_DISTRITO")))

lon: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 11 more fields]


In [13]:
lon.show()

+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
|COD_DISTRITO|       DESC_DISTRITO|COD_DIST_BARRIO|         DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|Longitud|
+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
|           1|CENTRO              |            101|PALACIO             |         1|            1001|          1|           0|               3|               1|                 1|              null|      20|
|           1|CENTRO              |            101|PALACIO             |         1|            1001|          1|           1|               5|               2|             

In [14]:
val col = lon.withColumn("Val5", lit(5))

col: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 12 more fields]


In [15]:
col.show()

+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+----+
|COD_DISTRITO|       DESC_DISTRITO|COD_DIST_BARRIO|         DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|Longitud|Val5|
+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+----+
|           1|CENTRO              |            101|PALACIO             |         1|            1001|          1|           0|               3|               1|                 1|              null|      20|   5|
|           1|CENTRO              |            101|PALACIO             |         1|            1001|          1|           1|               5|          

In [16]:
val del = col.drop("Val5")

del: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 11 more fields]


In [17]:
del.write.mode("overwrite")
    .partitionBy("DESC_DISTRITO","DESC_BARRIO")
    .csv("Particionado")

In [18]:
val numT =leer.select("DESC_DISTRITO","DESC_BARRIO","EspanolesHombres","EspanolesMujeres","ExtranjerosHombres","ExtranjerosMujeres")
    .groupBy("DESC_DISTRITO","DESC_BARRIO")
    .agg(sum("EspanolesHombres").as("EspanolesHombres")
      ,sum("EspanolesMujeres").as("EspanolesMujeres")
      ,sum("ExtranjerosHombres").as("ExtranjerosHombres")
      ,sum("ExtranjerosMujeres").as("ExtranjerosMujeres"))
    .orderBy(desc("ExtranjerosMujeres"),desc("ExtranjerosHombres"))

numT: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DESC_DISTRITO: string, DESC_BARRIO: string ... 4 more fields]


In [19]:
numT.show()

+--------------------+--------------------+----------------+----------------+------------------+------------------+
|       DESC_DISTRITO|         DESC_BARRIO|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+--------------------+--------------------+----------------+----------------+------------------+------------------+
|PUENTE DE VALLECAS  |SAN DIEGO           |           13889|           15566|              6935|              7096|
|CIUDAD LINEAL       |PUEBLO NUEVO        |           23112|           27183|              5626|              6415|
|LATINA              |ALUCHE              |           24824|           29557|              5451|              6386|
|CARABANCHEL         |VISTA ALEGRE        |           15791|           19255|              5553|              6175|
|CENTRO              |EMBAJADORES         |           16389|           16338|              7983|              5895|
|PUENTE DE VALLECAS  |NUMANCIA            |           17045|           1

In [20]:
val uni = leer
    .select(col("DESC_DISTRITO"), col("DESC_BARRIO"), col("EspanolesHombres"))
    .groupBy(col("DESC_DISTRITO"),col("DESC_BARRIO"))
    .agg(sum("EspanolesHombres").as("SumEspHombres"))

uni: org.apache.spark.sql.DataFrame = [DESC_DISTRITO: string, DESC_BARRIO: string ... 1 more field]


In [21]:
uni.show()

+--------------------+--------------------+-------------+
|       DESC_DISTRITO|         DESC_BARRIO|SumEspHombres|
+--------------------+--------------------+-------------+
|TETUAN              |VALDEACEDERAS       |         9495|
|ARGANZUELA          |DELICIAS            |        11746|
|CARABANCHEL         |VISTA ALEGRE        |        15791|
|SAN BLAS-CANILLEJAS |SIMANCAS            |        10689|
|ARGANZUELA          |LEGAZPI             |         8833|
|ARGANZUELA          |ATOCHA              |          744|
|MORATALAZ           |FONTARRON           |         6516|
|CARABANCHEL         |COMILLAS            |         8155|
|LATINA              |LOS CARMENES        |         6844|
|CENTRO              |CORTES              |         3945|
|PUENTE DE VALLECAS  |NUMANCIA            |        17045|
|RETIRO              |NI�O JESUS          |         6535|
|CIUDAD LINEAL       |PUEBLO NUEVO        |        23112|
|RETIRO              |PACIFICO            |        13285|
|MONCLOA-ARAVA

In [22]:
val join = leer.join(uni,leer("DESC_DISTRITO") === uni("DESC_DISTRITO") && leer("DESC_BARRIO") === uni("DESC_BARRIO"))
    .orderBy(col("COD_DISTRITO"),col("COD_DIST_BARRIO"))

join: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_DISTRITO: int, DESC_DISTRITO: string ... 13 more fields]


In [23]:
join.show()

+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------------------+--------------------+-------------+
|COD_DISTRITO|       DESC_DISTRITO|COD_DIST_BARRIO|         DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|       DESC_DISTRITO|         DESC_BARRIO|SumEspHombres|
+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------------------+--------------------+-------------+
|           1|CENTRO              |            101|PALACIO             |         1|            1001|          1|          18|               3|               7|                 2|              null|CENTRO              |PALACIO             

In [99]:
val pivot = leer
    .where(col("DESC_DISTRITO").equalTo("CENTRO") || col("DESC_DISTRITO").equalTo("BARAJAS") || col("DESC_DISTRITO").equalTo("RETIRO"))
    .groupBy(col("COD_EDAD_INT"))
    .pivot("DESC_DISTRITO").sum("EspanolesMujeres")
    .orderBy("COD_EDAD_INT")

pivot: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DESC_DISTRITO: string, COD_EDAD_INT: int]


In [97]:
pivot.show()

+-------------+------------+
|DESC_DISTRITO|COD_EDAD_INT|
+-------------+------------+
+-------------+------------+



In [ ]:
leer.write.mode("overwrite")
    .partitionBy("DESC_DISTRITO","DESC_BARRIO")
    .csv("salida/padron_csv")

In [ ]:
leer.write.mode("overwrite")
    .partitionBy("DESC_DISTRITO","DESC_BARRIO")
    .parquet("salida/padron_parquet")